# tokenization

토큰화 되는 squad_convert_examples_to_features 함수에 대한 설명 및 여기서 어떻게 토큰화 되는지에 대한 설명


In [1]:
from transformers.data.processors.squad import _is_whitespace, whitespace_tokenize, _improve_answer_span, _new_check_is_max_context

In [2]:
"""
1. 먼저 json 파일에서 파싱된 정보를 저장하고
2. 정답에 해당하는 answer text 를 가지고 end position 을 찾아 저장해둡니다
3. 공백 기준으로 토큰화를 진행해둡니다
""" 

class SquadExample:
    """
    A single training/test example for the Squad dataset, as loaded from disk.

    Args:
        qas_id: The example's unique identifier
        question_text: The question string
        context_text: The context string
        answer_text: The answer string
        start_position_character: The character position of the start of the answer
        title: The title of the example
        answers: None by default, this is used during evaluation. Holds answers as well as their start positions.
        is_impossible: False by default, set to True if the example has no possible answer.
    """

    def __init__(
        self,
        qas_id,
        question_text,
        context_text,
        answer_text,
        start_position_character,
        title,
        answers=[],
        is_impossible=False,
    ):
        self.qas_id = qas_id
        self.question_text = question_text
        self.context_text = context_text
        self.answer_text = answer_text
        self.title = title
        self.is_impossible = is_impossible
        self.answers = answers

        self.start_position, self.end_position = 0, 0

        doc_tokens = []
        char_to_word_offset = []
        prev_is_whitespace = True

        # Split on whitespace so that different tokens may be attributed to their original position.
        for c in self.context_text:
            if _is_whitespace(c):
                prev_is_whitespace = True
            else:
                if prev_is_whitespace:
                    doc_tokens.append(c)
                else:
                    doc_tokens[-1] += c
                prev_is_whitespace = False
            char_to_word_offset.append(len(doc_tokens) - 1)

        self.doc_tokens = doc_tokens
        self.char_to_word_offset = char_to_word_offset

        # Start and end positions only has a value during evaluation.
        if start_position_character is not None and not is_impossible:
            self.start_position = char_to_word_offset[start_position_character]
            self.end_position = char_to_word_offset[
                min(start_position_character + len(answer_text) - 1, len(char_to_word_offset) - 1)
            ]

In [3]:
# 예시로 저희 2주차 강의 예제 가져와볼게요
qas_id = "6521755-0-0"
q_text = "미국 군대 내 두번째로 높은 직위는 무엇인가?"
a_text = "미국 육군 부참모 총장"
start_position = 204
c_text = "알렉산더 메이그스 헤이그 2세(영어: Alexander Meigs Haig, Jr., 1924년 12월 2일 ~ 2010년 2월 20일)는 미국의 국무 장관을 지낸 미국의 군인, 관료 및 정치인이다. 로널드 레이건 대통령 밑에서 국무장관을 지냈으며, 리처드 닉슨과 제럴드 포드 대통령 밑에서 백악관 비서실장을 지냈다. 또한 그는 미국 군대에서 2번째로 높은 직위인 미국 육군 부참모 총장과 나토 및 미국 군대의 유럽연합군 최고사령관이었다. 한국 전쟁 시절 더글러스 맥아더 유엔군 사령관의 참모로 직접 참전하였으며, 로널드 레이건 정부 출범당시 초대 국무장관직을 맡아 1980년대 대한민국과 미국의 관계를 조율해 왔다. 저서로 회고록 《경고:현실주의, 레이건과 외교 정책》(1984년 발간)이 있다."
title = "알렉산더_헤이그"

example = SquadExample(qas_id, q_text, c_text, a_text, start_position, title)
print(example.start_position, example.end_position)

46 49


In [4]:
"""
이건 squad_convert_examples_to_features 에서 각 exmaple 을 처리하는 
squad_convert_example_to_features 함수 코드 입니다

"""
is_training = True

features = []
if is_training and not example.is_impossible:
    # Get start and end position
    start_position = example.start_position
    end_position = example.end_position

    # If the answer cannot be found in the text, then skip this example.
    actual_text = " ".join(example.doc_tokens[start_position : (end_position + 1)])
    cleaned_answer_text = " ".join(whitespace_tokenize(example.answer_text))
    if actual_text.find(cleaned_answer_text) == -1:
        print("Could not find answer: '%s' vs. '%s'", actual_text, cleaned_answer_text)
        
print(f"cleaend_answer_text, {cleaned_answer_text}")

cleaend_answer_text, 미국 육군 부참모 총장


In [5]:
"""
tokenizer 을 선언하고 공백기준 토큰들을 subword로 토큰화합니다
"""
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

tok_to_orig_index = []
orig_to_tok_index = []
all_doc_tokens = []
for (i, token) in enumerate(example.doc_tokens):
    orig_to_tok_index.append(len(all_doc_tokens))
    if tokenizer.__class__.__name__ in [
        "RobertaTokenizer",
        "LongformerTokenizer",
        "BartTokenizer",
        "RobertaTokenizerFast",
        "LongformerTokenizerFast",
        "BartTokenizerFast",
    ]:
        sub_tokens = tokenizer.tokenize(token, add_prefix_space=True)
    else:
        sub_tokens = tokenizer.tokenize(token)
    for sub_token in sub_tokens:
        tok_to_orig_index.append(i)
        all_doc_tokens.append(sub_token)

In [6]:
"""
subword 로 토큰을 바꾼 상태에서 start end position 을 맞추는 과정입니다
"""
if is_training and not example.is_impossible:
    tok_start_position = orig_to_tok_index[example.start_position]
    if example.end_position < len(example.doc_tokens) - 1:
        tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
    else:
        tok_end_position = len(all_doc_tokens) - 1

    (tok_start_position, tok_end_position) = _improve_answer_span(
        all_doc_tokens, tok_start_position, tok_end_position, tokenizer, example.answer_text
    )

In [7]:
len(all_doc_tokens)

225

In [12]:
"""
쿼리를 최대 길이에 맞게 자릅니다

sep 토큰을 넣고 config 에서 미리 설정한 전체 길이에 맞게 길이를 조정하도록 준비합니다
(sequence_pair_added_tokens, sequence_added_tokens)이 길이 조정 용도입니다
"""
spans = []
max_query_length = 32
# Store the tokenizers which insert 2 separators tokens
MULTI_SEP_TOKENS_TOKENIZERS_SET = {"roberta", "camembert", "bart", "mpnet"}

truncated_query = tokenizer.encode(
    example.question_text, add_special_tokens=False, truncation=True, max_length=max_query_length
)

# Tokenizers who insert 2 SEP tokens in-between <context> & <question> need to have special handling
# in the way they compute mask of added tokens.
tokenizer_type = type(tokenizer).__name__.replace("Tokenizer", "").lower()
sequence_added_tokens = (
    tokenizer.model_max_length - tokenizer.max_len_single_sentence + 1
    if tokenizer_type in MULTI_SEP_TOKENS_TOKENIZERS_SET
    else tokenizer.model_max_length - tokenizer.max_len_single_sentence
)
sequence_pair_added_tokens = tokenizer.model_max_length - tokenizer.max_len_sentences_pair

span_doc_tokens = all_doc_tokens


In [15]:
"""
미리 지정해둔 doc_stride 변수에 맞게 context 문서를 잘라냅니다
즉 하나의 context는 doc_stride 에 맞게
여러개의 sub_contexts으로 나뉩니다.
여기서는 answer 이 안들어가도 상관하지 않습니다 

"""
from transformers.tokenization_utils_base import TruncationStrategy
doc_stride= 64
padding_strategy = "max_length"
max_seq_length = 128

while len(spans) * doc_stride < len(all_doc_tokens):

    # Define the side we want to truncate / pad and the text/pair sorting
    if tokenizer.padding_side == "right":
        texts = truncated_query
        pairs = span_doc_tokens
        truncation = TruncationStrategy.ONLY_SECOND.value
    else:
        texts = span_doc_tokens
        pairs = truncated_query
        truncation = TruncationStrategy.ONLY_FIRST.value

    encoded_dict = tokenizer.encode_plus(  # TODO(thom) update this logic
        texts,
        pairs,
        truncation=truncation,
        padding=padding_strategy,
        max_length=max_seq_length,
        return_overflowing_tokens=True,
        stride=max_seq_length - doc_stride - len(truncated_query) - sequence_pair_added_tokens,
        return_token_type_ids=True,
    )

    paragraph_len = min(
        len(all_doc_tokens) - len(spans) * doc_stride,
        max_seq_length - len(truncated_query) - sequence_pair_added_tokens,
    )

    if tokenizer.pad_token_id in encoded_dict["input_ids"]:
        if tokenizer.padding_side == "right":
            non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
        else:
            last_padding_id_position = (
                len(encoded_dict["input_ids"]) - 1 - encoded_dict["input_ids"][::-1].index(tokenizer.pad_token_id)
            )
            non_padded_ids = encoded_dict["input_ids"][last_padding_id_position + 1 :]

    else:
        non_padded_ids = encoded_dict["input_ids"]

    tokens = tokenizer.convert_ids_to_tokens(non_padded_ids)

    token_to_orig_map = {}
    for i in range(paragraph_len):
        index = len(truncated_query) + sequence_added_tokens + i if tokenizer.padding_side == "right" else i
        token_to_orig_map[index] = tok_to_orig_index[len(spans) * doc_stride + i]

    encoded_dict["paragraph_len"] = paragraph_len
    encoded_dict["tokens"] = tokens
    encoded_dict["token_to_orig_map"] = token_to_orig_map
    encoded_dict["truncated_query_with_special_tokens_length"] = len(truncated_query) + sequence_added_tokens
    encoded_dict["token_is_max_context"] = {}
    encoded_dict["start"] = len(spans) * doc_stride
    encoded_dict["length"] = paragraph_len

    spans.append(encoded_dict)

    if "overflowing_tokens" not in encoded_dict or (
        "overflowing_tokens" in encoded_dict and len(encoded_dict["overflowing_tokens"]) == 0
    ):
        print("overflow")
        break
        
    span_doc_tokens = encoded_dict["overflowing_tokens"]


overflow


In [16]:
len(spans) # 3 개의 여러개의 subcontexts 으로 나뉩니다.

4

In [17]:
"""
체크 안한 부분 입니다
"""
for doc_span_index in range(len(spans)):
    for j in range(spans[doc_span_index]["paragraph_len"]):
        is_max_context = _new_check_is_max_context(spans, doc_span_index, doc_span_index * doc_stride + j)
        index = (
            j
            if tokenizer.padding_side == "left"
            else spans[doc_span_index]["truncated_query_with_special_tokens_length"] + j
        )
        spans[doc_span_index]["token_is_max_context"][index] = is_max_context


In [19]:
"""
여러개의 sub_context을 보면서 answer 이 들어가 있는지
등의 여부를 확인해 최종적인 feature 를 만들어냅니다
"""
import numpy as np
for span in spans:
    # Identify the position of the CLS token
    cls_index = span["input_ids"].index(tokenizer.cls_token_id)

    # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
    # Original TF implem also keep the classification token (set to 0)
    p_mask = np.ones_like(span["token_type_ids"])
    if tokenizer.padding_side == "right":
        p_mask[len(truncated_query) + sequence_added_tokens :] = 0
    else:
        p_mask[-len(span["tokens"]) : -(len(truncated_query) + sequence_added_tokens)] = 0

    pad_token_indices = np.where(span["input_ids"] == tokenizer.pad_token_id)
    special_token_indices = np.asarray(
        tokenizer.get_special_tokens_mask(span["input_ids"], already_has_special_tokens=True)
    ).nonzero()

    p_mask[pad_token_indices] = 1
    p_mask[special_token_indices] = 1

    # Set the cls index to 0: the CLS index can be used for impossible answers
    p_mask[cls_index] = 0

    span_is_impossible = example.is_impossible
    start_position = 0
    end_position = 0
    if is_training and not span_is_impossible:
        # For training, if our document chunk does not contain an annotation
        # we throw it out, since there is nothing to predict.
        doc_start = span["start"]
        doc_end = span["start"] + span["length"] - 1
        out_of_span = False

        if not (tok_start_position >= doc_start and tok_end_position <= doc_end):
            out_of_span = True

        if out_of_span:
            start_position = cls_index
            end_position = cls_index
            span_is_impossible = True
        else:
            if tokenizer.padding_side == "left":
                doc_offset = 0
            else:
                doc_offset = len(truncated_query) + sequence_added_tokens

            start_position = tok_start_position - doc_start + doc_offset
            end_position = tok_end_position - doc_start + doc_offset

    features.append(
        SquadFeatures(
            span["input_ids"],
            span["attention_mask"],
            span["token_type_ids"],
            cls_index,
            p_mask.tolist(),
            example_index=0,  # Can not set unique_id and example_index here. They will be set after multiple processing.
            unique_id=0,
            paragraph_len=span["paragraph_len"],
            token_is_max_context=span["token_is_max_context"],
            tokens=span["tokens"],
            token_to_orig_map=span["token_to_orig_map"],
            start_position=start_position,
            end_position=end_position,
            is_impossible=span_is_impossible,
            qas_id=example.qas_id,
        )
    )

NameError: name 'SquadFeatures' is not defined